# Second Order Lagrange Triangular Reference Element

## Import Packages

In [9]:
using LinearAlgebra # to define uniform scaling 
using Plots

In [10]:
include("quadrature.jl")

compute_weights_triangle

## Defining quadrature points and weights

In [12]:
# polynomial degree to FE approximation  
pdegree = 1;
# degree of numerical quadrature 
qorder = 1; 
xi = compute_coordinates_triangle(qorder)
w  = compute_weights_triangle(qorder)
display(xi)
display(w)

4-element Vector{Vector{Float64}}:
 [0.21132486540518708, 0.16666666666666663]
 [0.21132486540518708, 0.6220084679281462]
 [0.7886751345948129, 0.044658198738520435]
 [0.7886751345948129, 0.16666666666666663]

4-element Vector{Float64}:
 0.19716878364870322
 0.19716878364870322
 0.05283121635129677
 0.05283121635129677

## Computation on the Reference Element 

In [15]:
x1 = [0.,0.]; x2 = [1.,0.]; x3 = [0.,1.];  
x4 = (x1+x2)/2; x5 = (x2+x3)/2; x6 = (x1+x3)/2;  

#....compute surface area of the current element
area_id = 0.5 

#....computation of the coefficients of the basis functions 
x = [x1, x2, x3, x4, x5, x6]
X = [[x1[1]^2, x1[2]^2, x1[1]*x1[2], x1[1], x1[2], 1] for x1 in x]
X = reduce(vcat,transpose(X))
Emat = X\UniformScaling(1.)
   
#....transform quadrature points to physical domain    
T = zeros(2,2); T[:,1] = x2 - x1; T[:,2] = x3 - x1; 
xs = [T*xi1+x1 for xi1 in xi];

#....construct Vandermonde Matrix for evaluation of the basis functions
Xs = zeros(length(w),6);   
Xs = [[xs1[1]^2, xs1[2]^2, xs1[1]*xs1[2], xs1[1], xs1[2], 1] for xs1 in xs];
Xs = reduce(vcat,transpose(Xs))
    
#....construct Vandermonde Matrix for evaluation of the x/y derivatives of basis functions
dxXs = zeros(length(w),6);
dxXs = [[2*xs1[1], 0 , xs1[2], 1, 0, 0] for xs1 in xs];
dxXs = reduce(vcat,transpose(dxXs))
dyXs = zeros(length(w),6);
dyXs = [[0, 2*xs1[2], xs1[1], 0, 1, 0] for xs1 in xs];
dyXs = reduce(vcat,transpose(dyXs))
    
#....coonstruct column matrices of basis functions and derivatives of the basis functions   
Phi   = Xs*Emat; 
dxPhi = dxXs*Emat; 
dyPhi = dyXs*Emat; 

#....compute contributions to the local mass and stiffness matrix   
PhiPhi = zeros(length(w),36);
[PhiPhi[:,(j-1)*6+i] = Phi[:,i].*Phi[:,j] for i=1:6,j=1:6];
dxPhidxPhi = zeros(length(w),36);
[dxPhidxPhi[:,(j-1)*6+i] = dxPhi[:,i].*dxPhi[:,j] for i=1:6,j=1:6];
dyPhidyPhi = zeros(length(w),36);
[dyPhidyPhi[:,(j-1)*6+i] = dyPhi[:,i].*dyPhi[:,j] for i=1:6,j=1:6];
dPhidPhi = dxPhidxPhi + dyPhidyPhi; 
Mloc = 2*area_id*Transpose(w)*PhiPhi
Mloc = reshape(Mloc,6,6)
Aloc = 2*area_id*Transpose(w)*dPhidPhi
Aloc = reshape(Aloc,6,6)

print("Matrix Emat definining basis functions")
display(Emat)
print("Elementary stiffness matrix")
display(Aloc)
print("Elementary mass matrix")
display(Mloc)

Matrix Emat definining basis functions

6×6 Matrix{Float64}:
  2.0   2.0   0.0  -4.0  0.0  -0.0
  2.0   0.0   2.0   0.0  0.0  -4.0
  4.0   0.0   0.0  -4.0  4.0  -4.0
 -3.0  -1.0   0.0   4.0  0.0   0.0
 -3.0   0.0  -1.0   0.0  0.0   4.0
  1.0   0.0   0.0   0.0  0.0   0.0

Elementary stiffness matrix

6×6 reshape(transpose(::Vector{Float64}), 6, 6) with eltype Float64:
  1.0           0.166667      0.166667     …  -4.44089e-16  -0.666667
  0.166667      0.5           0.0             -1.41031e-16   1.41031e-16
  0.166667      0.0           0.5             -2.77556e-17  -0.666667
 -0.666667     -0.666667      2.77556e-17     -1.33333      -2.22045e-16
 -4.44089e-16  -1.41031e-16  -2.77556e-17      2.66667      -1.33333
 -0.666667      1.41031e-16  -0.666667     …  -1.33333       2.66667

Elementary mass matrix

6×6 reshape(transpose(::Vector{Float64}), 6, 6) with eltype Float64:
  0.00771605  -0.00462963   -0.00617284  …  -0.00925926    0.00308642
 -0.00462963   0.0277778    -0.00462963      9.56591e-19  -0.0185185
 -0.00617284  -0.00462963    0.00771605      0.00925926    0.00308642
  0.00925926  -4.51594e-18  -0.00925926      0.037037      0.0555556
 -0.00925926   9.56591e-19   0.00925926      0.0740741     0.0555556
  0.00308642  -0.0185185     0.00308642  …   0.0555556     0.0679012